In [17]:
import pandas as pd
from transformers import pipeline
import numpy as np
from tqdm import tqdm

In [5]:
books = pd.read_csv("../processed_data/books_with_categories.csv")

In [7]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critic...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' ...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroin...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lo...,NonFiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. L...",NonFiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To Nort...,NonFiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a m...,NonFiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Pass...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless...,Nonfiction


In [11]:
classifier = pipeline(
    "text-classification", 
    model="j-hartmann/emotion-english-distilroberta-base", 
    top_k = None,
    device= 'mps'
)


Device set to use mps


In [15]:
books["description"]

0       A NOVEL THAT READERS and critics have been eag...
1       A new 'Christie for Christmas' -- a full-lengt...
2       A memorable, mesmerizing heroine Jennifer -- b...
3       Lewis' work on the nature of love divides love...
4       "In The Problem of Pain, C.S. Lewis, one of th...
                              ...                        
5192    On A Train Journey Home To North India After L...
5193    This book tells the tale of a man who goes on ...
5194    Wisdom to Create a Life of Passion, Purpose, a...
5195    This collection of the timeless teachings of o...
5196    Since the three volume edition ofHegel's Philo...
Name: description, Length: 5197, dtype: object

In [12]:
classifier("I love this")

[[{'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'surprise', 'score': 0.0049272081814706326},
  {'label': 'sadness', 'score': 0.004531430080533028},
  {'label': 'neutral', 'score': 0.0034753174986690283},
  {'label': 'anger', 'score': 0.0013875322183594108},
  {'label': 'disgust', 'score': 0.0007134054903872311},
  {'label': 'fear', 'score': 0.0003984911891166121}]]

In [16]:
classifier(books["description"][0])
# doesn’t capture the mix of emotions in the text.

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.1698521226644516},
  {'label': 'sadness', 'score': 0.11640863865613937},
  {'label': 'surprise', 'score': 0.020700639113783836},
  {'label': 'disgust', 'score': 0.019100764766335487},
  {'label': 'joy', 'score': 0.015161274001002312},
  {'label': 'anger', 'score': 0.003935154527425766}]]

In [14]:
# Instead of classifying the entire description, split into sentences.
# Feed sentences individually into the classifier (many models can handle multiple sentences at once).
# As we see first sentence-> surprise, second sentnece-> neutral

classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.1403856724500656},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.04794240742921829},
  {'label': 'anger', 'score': 0.009156349115073681},
  {'label': 'disgust', 'score': 0.002628474263474345},
  {'label': 'sadness', 'score': 0.0021221607457846403}],
 [{'label': 'neutral', 'score': 0.449371874332428},
  {'label': 'disgust', 'score': 0.27359071373939514},
  {'label': 'joy', 'score': 0.10908263921737671},
  {'label': 'sadness', 'score': 0.09362728893756866},
  {'label': 'anger', 'score': 0.040478210896253586},
  {'label': 'surprise', 'score': 0.026970218867063522},
  {'label': 'fear', 'score': 0.006879058200865984}],
 [{'label': 'neutral', 'score': 0.6462154388427734},
  {'label': 'sadness', 'score': 0.24273407459259033},
  {'label': 'disgust', 'score': 0.043422624468803406},
  {'label': 'surprise', 'score': 0.028300493955612183},
  {'label': 'joy', 'score': 0.0142114

In [18]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [19]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [20]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'sadness', 'score': 0.0021221607457846403}]

In [21]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [22]:
predictions[3]

[{'label': 'fear', 'score': 0.9281686544418335},
 {'label': 'anger', 'score': 0.03219051659107208},
 {'label': 'neutral', 'score': 0.012808646075427532},
 {'label': 'sadness', 'score': 0.008756857365369797},
 {'label': 'surprise', 'score': 0.008597849868237972},
 {'label': 'disgust', 'score': 0.008431794121861458},
 {'label': 'joy', 'score': 0.0010455797892063856}]

 so from the above we see multiple emotions for the same book cause per sentence the emotion changes. so what we can do is for each book have a seperate column for each of the 7 classes. 

In [23]:
predictions

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.1403856724500656},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.04794240742921829},
  {'label': 'anger', 'score': 0.009156349115073681},
  {'label': 'disgust', 'score': 0.002628474263474345},
  {'label': 'sadness', 'score': 0.0021221607457846403}],
 [{'label': 'neutral', 'score': 0.449371874332428},
  {'label': 'disgust', 'score': 0.27359071373939514},
  {'label': 'joy', 'score': 0.10908263921737671},
  {'label': 'sadness', 'score': 0.09362728893756866},
  {'label': 'anger', 'score': 0.040478210896253586},
  {'label': 'surprise', 'score': 0.026970218867063522},
  {'label': 'fear', 'score': 0.006879058200865984}],
 [{'label': 'neutral', 'score': 0.6462154388427734},
  {'label': 'sadness', 'score': 0.24273407459259033},
  {'label': 'disgust', 'score': 0.043422624468803406},
  {'label': 'surprise', 'score': 0.028300493955612183},
  {'label': 'joy', 'score': 0.0142114

In [24]:
# Take the highest probability (max score) found for each emotion across all sentences.
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'sadness', 'score': 0.0021221607457846403},
 {'label': 'surprise', 'score': 0.7296027541160583}]

In [25]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
# Create a dictionary comprehension → holds predictions for a single description.
emotion_scores = {label: [] for label in emotion_labels}

In [26]:
def calculate_max_emotion_scores(predictions):
    # Create a dictionary comprehension → holds predictions for a single description.
    per_emotion_scores = {label: [] for label in emotion_labels}
    # Each book description is split into sentences.
    for prediction in predictions:
        #Sort the predictions by label → ensures same label order across sentences.
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        # Extract scores per emotion
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

# for each book get the scores we test it out for 10 books

In [27]:
# 
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])


In [28]:
emotion_scores

{'anger': [np.float64(0.06413356214761734),
  np.float64(0.612618625164032),
  np.float64(0.06413356214761734),
  np.float64(0.35148370265960693),
  np.float64(0.08141237497329712),
  np.float64(0.23222501575946808),
  np.float64(0.538184642791748),
  np.float64(0.06413356214761734),
  np.float64(0.3006702959537506),
  np.float64(0.06413356214761734)],
 'disgust': [np.float64(0.27359071373939514),
  np.float64(0.3482856750488281),
  np.float64(0.10400660336017609),
  np.float64(0.15072238445281982),
  np.float64(0.1844952255487442),
  np.float64(0.727174699306488),
  np.float64(0.15585507452487946),
  np.float64(0.10400660336017609),
  np.float64(0.2794807255268097),
  np.float64(0.1779274344444275)],
 'fear': [np.float64(0.9281686544418335),
  np.float64(0.9425278306007385),
  np.float64(0.9723208546638489),
  np.float64(0.3607065975666046),
  np.float64(0.09504325687885284),
  np.float64(0.05136275663971901),
  np.float64(0.7474279999732971),
  np.float64(0.4044961929321289),
  np.fl

# do it for all books in the dataset

In [29]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
#Dictionary where each emotion is a key, and values will be a list of scores (one per book).
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [1:26:43<00:00,  1.00s/it]


In [30]:
# pass the results a pandas dataframe
emotions_df = pd.DataFrame(emotion_scores)
# create a speate column for isbn and assign isbn list to that
emotions_df["isbn13"] = isbn

In [31]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646215,0.967158,0.729603,9780002005883
1,0.612619,0.348286,0.942528,0.704421,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339216,0.947779,0.375756,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368110,0.078765,9788185300535


In [32]:
# merge it to the original dataframe
books = pd.merge(books, emotions_df, on = "isbn13")

In [33]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critic...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646215,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' ...,Fiction,0.612619,0.348286,0.942528,0.704421,0.887939,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroin...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lo...,NonFiction,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. L...",NonFiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To Nort...,NonFiction,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a m...,NonFiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Pass...,Fiction,0.009997,0.009929,0.339216,0.947779,0.375756,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368110,0.078765


In [35]:
books.to_csv("../processed_data/books_with_emotions.csv", index = False)